In [5]:
import os
PROJECT = "PROJECT" # REPLACE WITH YOUR PROJECT ID
BUCKET = "BUCKET" # REPLACE WITH A BUCKET NAME (PUT YOUR PROJECT ID AND WE CREATE THE BUCKET ITSELF NEXT)
REGION = "us-east1" # REPLACE WITH YOUR REGION e.g. us-central1

# Import os environment variables
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] =  BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "1.13"

## Now write into a python module

In [ ]:
%%writefile lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer/model.py
import tensorflow as tf

# Set logging to be level of INFO
tf.logging.set_verbosity(tf.logging.INFO)

# Determine CSV and label columns
number_of_tags = 5
tag_columns = ["tag_{0}".format(tag) for tag in range(0, number_of_tags)]
UNLABELED_CSV_COLUMNS = tag_columns

LABEL_COLUMN = "anomalous_sequence_flag"
LABELED_CSV_COLUMNS = UNLABELED_CSV_COLUMNS + [LABEL_COLUMN]

# Set default values for each CSV column
UNLABELED_DEFAULTS = [[""] for _ in UNLABELED_CSV_COLUMNS]

LABELED_DEFAULTS = UNLABELED_DEFAULTS + [[0.0]]

# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename, mode, batch_size, params):
  def _input_fn():
    def decode_csv(value_column, seq_len):
      def convert_sequences_from_strings_to_floats(features, column_list):
        def split_and_convert_string(string_tensor):
          # Split string tensor into a sparse tensor based on delimiter
          split_string = tf.string_split(source = tf.expand_dims(
            input = string_tensor, axis = 0), delimiter = ",")

          # Converts the values of the sparse tensor to floats
          converted_tensor = tf.string_to_number(
            string_tensor = split_string.values, 
            out_type = tf.float64)

          # Create a new sparse tensor with the new converted values, 
          # because the original sparse tensor values are immutable
          new_sparse_tensor = tf.SparseTensor(
            indices = split_string.indices, 
            values = converted_tensor, 
            dense_shape = split_string.dense_shape)

          # Create a dense tensor of the float values that were converted from text csv
          dense_floats = tf.sparse_tensor_to_dense(
            sp_input = new_sparse_tensor, default_value = 0.0)

          dense_floats_vector = tf.squeeze(input = dense_floats, axis = 0)

          return dense_floats_vector
          
        for column in column_list:
          features[column] = split_and_convert_string(features[column])
          features[column].set_shape([seq_len])

        return features
        
      if mode == tf.estimator.ModeKeys.TRAIN or (mode == tf.estimator.ModeKeys.EVAL and params["evaluation_mode"] != "tune_anomaly_thresholds"):
        columns = tf.decode_csv(
          records = value_column, 
          record_defaults = UNLABELED_DEFAULTS, 
          field_delim = ";")
        features = dict(zip(UNLABELED_CSV_COLUMNS, columns))
        features = convert_sequences_from_strings_to_floats(
          features, UNLABELED_CSV_COLUMNS)
        return features
      else:
        columns = tf.decode_csv(
          records = value_column, 
          record_defaults = LABELED_DEFAULTS, 
          field_delim = ";")
        features = dict(zip(LABELED_CSV_COLUMNS, columns))
        labels = tf.cast(x = features.pop(LABEL_COLUMN), dtype = tf.float64)
        features = convert_sequences_from_strings_to_floats(
          features, LABELED_CSV_COLUMNS[0:-1])
        return features, labels
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename = filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(filenames = file_list)  # Read text file

    # Decode the CSV file into a features dictionary of tensors
    dataset = dataset.map(map_func = lambda x: decode_csv(x, params["seq_len"]))
    
    # Determine amount of times to repeat file based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # indefinitely
    else:
      num_epochs = 1 # end-of-input after this

    # Repeat files num_epoch times
    dataset = dataset.repeat(count = num_epochs)

    # Group the data into batches
    dataset = dataset.batch(batch_size = batch_size)
    
    # Determine if we should shuffle based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      dataset = dataset.shuffle(buffer_size = 10 * batch_size)

    # Create a iterator and then pull the next batch of features from the example queue
    batched_dataset = dataset.make_one_shot_iterator().get_next()

    return batched_dataset
  return _input_fn

def create_LSTM_stack(lstm_hidden_units, lstm_dropout_output_keep_probs):
  # First create a list of LSTM cells using our list of lstm hidden unit sizes
  lstm_cells = [tf.contrib.rnn.BasicLSTMCell(
    num_units = units, 
    forget_bias = 1.0, 
    state_is_tuple = True) for units in lstm_hidden_units] # list of LSTM cells

  # Next apply a dropout wrapper to our stack of LSTM cells, in this case just on the outputs
  dropout_lstm_cells = [tf.nn.rnn_cell.DropoutWrapper(
    cell = lstm_cells[cell_index], 
    input_keep_prob = 1.0, 
    output_keep_prob = lstm_dropout_output_keep_probs[cell_index], 
    state_keep_prob = 1.0) for cell_index in range(len(lstm_cells))]

  # Create a stack of layers of LSTM cells
  stacked_lstm_cells = tf.contrib.rnn.MultiRNNCell(
    cells = dropout_lstm_cells, 
    state_is_tuple = True) # combines list into MultiRNNCell object

  return stacked_lstm_cells

# The rnn_decoder function takes labels during TRAIN/EVAL 
# and a start token followed by its previous predictions during PREDICT
# Starts with an intial state of the final encoder states
def rnn_decoder(decoder_inputs, initial_state, cell, inference, dnn_hidden_units, num_features):
  # Create the decoder variable scope
  with tf.variable_scope("decoder"):
    # Load in our initial state from our encoder
    state = initial_state # tuple of final encoder c_state and h_state of final encoder layer

    # Create an empty list to store our hidden state output for every timestep
    outputs = []

    # Begin with no previous output
    previous_output = None

    # Loop over all of our decoder_inputs which will be seq_len long
    for index, decoder_input in enumerate(decoder_inputs):
      # If there has been a previous output then we will determine the next input
      if previous_output is not None:
        # Create the input layer to our DNN
        network = previous_output # shape = (cur_batch_size, lstm_hidden_units[-1])

        # Create our dnn variable scope
        with tf.variable_scope(name_or_scope = "dnn", reuse = tf.AUTO_REUSE):
          # Add hidden layers with the given number of units/neurons per layer
          # shape = (cur_batch_size, dnn_hidden_units[i])
          for units in dnn_hidden_units:
            network = tf.layers.dense(
              inputs = network, 
              units = units, 
              activation = tf.nn.relu)

          # Connect final hidden layer to linear layer to get the logits
          logits = tf.layers.dense(
            inputs = network, 
            units = num_features, 
            activation = None) # shape = (cur_batch_size, num_features)

        # If we are in inference then we will overwrite our next decoder_input 
        # with the logits we just calculated.
        # Otherwise, we leave the decoder_input input as it was from the enumerated list
        # We have to calculate the logits even when not using them so that the correct 
        # dnn subgraph will be generated here and after the encoder-decoder for both 
        # training and inference
        if inference == True:
          decoder_input = logits # shape = (cur_batch_size, num_features)

      # If this isn"t our first time through the loop, just reuse(share) the same 
      # variables for each iteration within the current variable scope
      if index > 0:
        tf.get_variable_scope().reuse_variables()

      # Run the decoder input through the decoder stack picking up from the previous state
      # output_shape = (cur_batch_size, lstm_hidden_units[-1])
      # state_shape = # tuple of final decoder c_state and h_state
      output, state = cell(decoder_input, state)

      # Append the current decoder hidden state output to the outputs list
      # list eventually seq_len long of shape = (cur_batch_size, lstm_hidden_units[-1])
      outputs.append(output)

      # Set the previous output to the output just calculated
      previous_output = output # shape = (cur_batch_size, lstm_hidden_units[-1])
  return outputs, state

# This function updates the count of records used
def update_count(count_a, count_b):
  return count_a + count_b

# This function updates the mahalanobis distance variables when number_of_rows equals 1
def singleton_batch_mahalanobis_distance_variable_updating(
  inner_size, 
  X, 
  count_variable, 
  mean_variable, 
  cov_variable, 
  inverse_cov_variable,
  eps):
  # This function updates the mean vector incrementally
  def update_mean_incremental(count_a, mean_a, value_b):
    mean_ab = (mean_a * tf.cast(x = count_a, dtype = tf.float64) + \
           tf.squeeze(input = value_b, axis = 0)) / tf.cast(x = count_a + 1, dtype = tf.float64)
    return mean_ab

  # This function updates the covariance matrix incrementally
  def update_cov_incremental(count_a, mean_a, cov_a, value_b, mean_ab, sample_cov):
    if sample_cov == True:
      cov_ab = (cov_a * tf.cast(x = count_a - 1, dtype = tf.float64) + \
            tf.matmul(a = value_b - mean_a, b = value_b - mean_ab, transpose_a = True)) \
        / tf.cast(x = count_a, dtype = tf.float64)
    else:
      cov_ab = (cov_a * tf.cast(x = count_a, dtype = tf.float64) + \
            tf.matmul(a = value_b - mean_a, b = value_b - mean_ab, transpose_a = True)) \
        / tf.cast(x = count_a + 1, dtype = tf.float64)
    return cov_ab

  # Calculate new combined mean to use for incremental covariance matrix calculation
  mean_ab = update_mean_incremental(
    count_a = count_variable, 
    mean_a = mean_variable, 
    value_b = X) # time_shape = (num_features,), features_shape = (sequence_length,)

  # Update running variables from single example
  count_tensor = update_count(
    count_a = count_variable, 
    count_b = 1) # time_shape = (), features_shape = ()

  mean_tensor = mean_ab # time_shape = (num_features,), features_shape = (sequence_length,)

  if inner_size == 1:
    cov_tensor = tf.zeros_like(
      tensor = cov_variable, dtype = tf.float64)
    inverse_cov_tensor = tf.eye(
      num_rows = tf.shape(
        input = cov_tensor)[0], 
        dtype = tf.float64) / eps
  else:
    # time_shape = (num_features, num_features)
    # features_shape = (sequence_length, sequence_length)
    cov_tensor = update_cov_incremental(
      count_a = count_variable, 
      mean_a = mean_variable, 
      cov_a = cov_variable, 
      value_b = X, 
      mean_ab = mean_ab, 
      sample_cov = True)

    # time_shape = (num_features, num_features)
    # features_shape = (sequence_length, sequence_length)
    inverse_cov_tensor = tf.matrix_inverse(
      input = cov_tensor + tf.eye(
        num_rows = tf.shape(
          input = cov_tensor)[0], 
          dtype = tf.float64) * eps)

  # Assign values to variables, use control dependencies around return to enforce the mahalanobis 
  # variables to be assigned, the control order matters, hence the separate contexts
  with tf.control_dependencies(
    control_inputs = [tf.assign(
      ref = cov_variable, 
      value = cov_tensor)]):
    with tf.control_dependencies(
      control_inputs = [tf.assign(
        ref = mean_variable, 
        value = mean_tensor)]):
      with tf.control_dependencies(
        control_inputs = [tf.assign(
          ref = count_variable, 
          value = count_tensor)]):
        with tf.control_dependencies(
          control_inputs = [tf.assign(
            ref = inverse_cov_variable, 
            value = inverse_cov_tensor)]):
          return tf.identity(input = cov_variable), tf.identity(input = mean_variable), tf.identity(input = count_variable), tf.identity(input = inverse_cov_variable)

# This function updates the mahalanobis distance variables when number_of_rows does NOT equal 1
def non_singleton_batch_mahalanobis_distance_variable_updating(
  cur_batch_size, 
  inner_size, 
  X, 
  count_variable, 
  mean_variable, 
  cov_variable, 
  inverse_cov_variable,
  eps):
  # This function updates the mean vector using a batch of data
  def update_mean_batch(count_a, mean_a, count_b, mean_b):
    mean_ab = (mean_a * tf.cast(x = count_a, dtype = tf.float64) + \
               mean_b * tf.cast(x = count_b, dtype = tf.float64)) \
               / tf.cast(x = count_a + count_b, dtype = tf.float64)
    return mean_ab

  # This function updates the covariance matrix using a batch of data
  def update_cov_batch(count_a, mean_a, cov_a, count_b, mean_b, cov_b, sample_cov):
    mean_diff = tf.expand_dims(input = mean_a - mean_b, axis = 0)

    if sample_cov == True:
      cov_ab = (cov_a * tf.cast(x = count_a - 1, dtype = tf.float64) + \
                cov_b * tf.cast(x = count_b - 1, dtype = tf.float64) + \
                tf.matmul(a = mean_diff, b = mean_diff, transpose_a = True) * \
                tf.cast(x = count_a * count_b, dtype = tf.float64) \
                / tf.cast(x = count_a + count_b, dtype = tf.float64)) \
                / tf.cast(x = count_a + count_b - 1, dtype = tf.float64)
    else:
      cov_ab = (cov_a * tf.cast(x = count_a, dtype = tf.float64) + \
                cov_b * tf.cast(x = count_b, dtype = tf.float64) + \
                tf.matmul(a = mean_diff, b = mean_diff, transpose_a = True) * \
                tf.cast(x = count_a * count_b, dtype = tf.float64) \
                / tf.cast(x = count_a + count_b, dtype = tf.float64)) \
                / tf.cast(x = count_a + count_b, dtype = tf.float64)
    return cov_ab          

  # Find statistics of batch
  number_of_rows = cur_batch_size * inner_size

  # time_shape = (num_features,), features_shape = (sequence_length,)
  X_mean = tf.reduce_mean(input_tensor = X, axis = 0)

  # time_shape = (cur_batch_size * sequence_length, num_features)
  # features_shape = (cur_batch_size * num_features, sequence_length)
  X_centered = X - X_mean

  if inner_size > 1:
    # time_shape = (num_features, num_features)
    # features_shape = (sequence_length, sequence_length)
    X_cov = tf.matmul(
      a = X_centered,
      b = X_centered, 
      transpose_a = True) / tf.cast(x = number_of_rows - 1, dtype = tf.float64)

  # Update running variables from batch statistics
  count_tensor = update_count(
    count_a = count_variable, 
    count_b = number_of_rows) # time_shape = (), features_shape = ()

  mean_tensor = update_mean_batch(
    count_a = count_variable, 
    mean_a = mean_variable, 
    count_b = number_of_rows, 
    mean_b = X_mean) # time_shape = (num_features,), features_shape = (sequence_length,)

  if inner_size == 1:
    cov_tensor = tf.zeros_like(
      tensor = cov_variable, dtype = tf.float64)
    inverse_cov_tensor = tf.eye(
      num_rows = tf.shape(input = cov_tensor)[0], dtype = tf.float64) / eps
  else:
    # time_shape = (num_features, num_features)
    # features_shape = (sequence_length, sequence_length)
    cov_tensor = update_cov_batch(
      count_a = count_variable, 
      mean_a = mean_variable, 
      cov_a = cov_variable, 
      count_b = number_of_rows, 
      mean_b = X_mean, 
      cov_b = X_cov, 
      sample_cov = True)

    # time_shape = (num_features, num_features)
    # features_shape = (sequence_length, sequence_length)
    inverse_cov_tensor = tf.matrix_inverse(
      input = cov_tensor + \
      tf.eye(num_rows = tf.shape(input = cov_tensor)[0], 
      dtype = tf.float64) * eps)

  # Assign values to variables, use control dependencies around return to enforce the mahalanobis 
  # variables to be assigned, the control order matters, hence the separate contexts
  with tf.control_dependencies(
    control_inputs = [tf.assign(ref = cov_variable, value = cov_tensor)]):
    with tf.control_dependencies(
      control_inputs = [tf.assign(ref = mean_variable, value = mean_tensor)]):
      with tf.control_dependencies(
        control_inputs = [tf.assign(ref = count_variable, value = count_tensor)]):
        with tf.control_dependencies(
          control_inputs = [tf.assign(ref = inverse_cov_variable, value = inverse_cov_tensor)]):
          return tf.identity(input = cov_variable), tf.identity(input = mean_variable), tf.identity(input = count_variable), tf.identity(input = inverse_cov_variable)
        
def mahalanobis_distance(error_vectors_reshaped, mean_vector, inv_covariance, final_shape):
  # time_shape = (current_batch_size * seq_len, num_features)
  # features_shape = (current_batch_size * num_features, seq_len)
  error_vectors_reshaped_centered = error_vectors_reshaped - mean_vector

  # time_shape = (num_features, current_batch_size * seq_len)
  # features_shape = (seq_len, current_batch_size * num_features)
  mahalanobis_right_product = tf.matmul(
    a = inv_covariance,
    b = error_vectors_reshaped_centered,
    transpose_b = True)

  # time_shape = (current_batch_size * seq_len, current_batch_size * seq_len)
  # features_shape = (current_batch_size * num_features, current_batch_size * num_features)
  mahalanobis_distance_vectorized = tf.matmul(
    a = error_vectors_reshaped_centered,
    b = mahalanobis_right_product)

  # time_shape = (current_batch_size * seq_len,)
  # features_shape = (current_batch_size * num_features,)
  mahalanobis_distance_flat = tf.diag_part(input = mahalanobis_distance_vectorized)

  # time_shape = (current_batch_size, seq_len)
  # features_shape = (current_batch_size, num_features)
  mahalanobis_distance_final_shaped = tf.reshape(
    tensor = mahalanobis_distance_flat, 
    shape = [-1, final_shape])

  # time_shape = (current_batch_size, seq_len)
  # features_shape = (current_batch_size, num_features)
  mahalanobis_distance_final_shaped_abs = tf.abs(x = mahalanobis_distance_final_shaped)

  return mahalanobis_distance_final_shaped_abs

def update_anomaly_threshold_variables(
  labels_normal_mask, 
  labels_anomalous_mask, 
  num_thresholds, 
  anomaly_thresholds, 
  mahalanobis_distance, 
  tp_at_thresholds_variable, 
  fn_at_thresholds_variable, 
  fp_at_thresholds_variable, 
  tn_at_thresholds_variable,
  mode):
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    # time_shape = (num_time_anomaly_thresholds, current_batch_size, sequence_length)
    # features_shape = (num_features_anomaly_thresholds, current_batch_size, number_of_features)
    mahalanobis_distance_over_thresholds = tf.map_fn(
      fn = lambda anomaly_threshold: mahalanobis_distance > anomaly_threshold, 
      elems = anomaly_thresholds, 
      dtype = tf.bool)
  else:
    # time_shape = (current_batch_size, sequence_length)
    # features_shape = (current_batch_size, number_of_features)
    mahalanobis_distance_over_thresholds = mahalanobis_distance > anomaly_thresholds

  # time_shape = (num_time_anomaly_thresholds, current_batch_size)
  # features_shape = (num_features_anomaly_thresholds, current_batch_size)    
  mahalanobis_distance_any_over_thresholds = tf.reduce_any(
    input_tensor = mahalanobis_distance_over_thresholds, 
    axis = -1)
    
  if mode == tf.estimator.ModeKeys.EVAL:
    # time_shape = (1, current_batch_size)
    # features_shape = (1, current_batch_size)
    mahalanobis_distance_any_over_thresholds = tf.expand_dims(
      input = mahalanobis_distance_any_over_thresholds, axis = 0)

  # time_shape = (num_time_anomaly_thresholds, current_batch_size)
  # features_shape = (num_features_anomaly_thresholds, current_batch_size)
  predicted_normals = tf.equal(
    x = mahalanobis_distance_any_over_thresholds, 
    y = False)

  # time_shape = (num_time_anomaly_thresholds, current_batch_size)
  # features_shape = (num_features_anomaly_thresholds, current_batch_size)
  predicted_anomalies = tf.equal(
    x = mahalanobis_distance_any_over_thresholds, 
    y = True)
  
  # Calculate confusion matrix of current batch
  # time_shape = (num_time_anomaly_thresholds,)
  # features_shape = (num_features_anomaly_thresholds,)
  tp = tf.reduce_sum(
    input_tensor = tf.cast(
      x = tf.map_fn(
        fn = lambda threshold: tf.logical_and(
          x = labels_anomalous_mask, 
          y = predicted_anomalies[threshold, :]), 
        elems = tf.range(start = 0, limit = num_thresholds, dtype = tf.int64), 
        dtype = tf.bool), 
      dtype = tf.int64), 
    axis = 1)

  fn = tf.reduce_sum(
    input_tensor = tf.cast(
      x = tf.map_fn(
        fn = lambda threshold: tf.logical_and(
          x = labels_anomalous_mask, 
          y = predicted_normals[threshold, :]), 
        elems = tf.range(start = 0, limit = num_thresholds, dtype = tf.int64), 
        dtype = tf.bool), 
      dtype = tf.int64), 
    axis = 1)

  fp = tf.reduce_sum(
    input_tensor = tf.cast(
      x = tf.map_fn(
        fn = lambda threshold: tf.logical_and(
          x = labels_normal_mask, 
          y = predicted_anomalies[threshold, :]), 
        elems = tf.range(start = 0, limit = num_thresholds, dtype = tf.int64), 
        dtype = tf.bool), 
      dtype = tf.int64), 
    axis = 1)

  tn = tf.reduce_sum(
    input_tensor = tf.cast(
      x = tf.map_fn(
        fn = lambda threshold: tf.logical_and(
          x = labels_normal_mask, 
          y = predicted_normals[threshold, :]), 
        elems = tf.range(start = 0, limit = num_thresholds, dtype = tf.int64), 
        dtype = tf.bool), 
      dtype = tf.int64), 
    axis = 1)
  
  if mode == tf.estimator.ModeKeys.EVAL:
    # shape = ()
    tp = tf.squeeze(input = tp)
    fn = tf.squeeze(input = fn)
    fp = tf.squeeze(input = fp)
    tn = tf.squeeze(input = tn)

  with tf.control_dependencies(
    control_inputs = [tf.assign_add(ref = tp_at_thresholds_variable, value = tp), 
                      tf.assign_add(ref = fn_at_thresholds_variable, value = fn), 
                      tf.assign_add(ref = fp_at_thresholds_variable, value = fp), 
                      tf.assign_add(ref = tn_at_thresholds_variable, value = tn)]):
    return tf.identity(input = tp_at_thresholds_variable), tf.identity(input = fn_at_thresholds_variable), tf.identity(input = fp_at_thresholds_variable), tf.identity(input = tn_at_thresholds_variable)
  
def calculate_composite_classification_metrics(anomaly_thresholds, tp, fn, fp, tn, f_score_beta):
  # time_shape = (num_time_anomaly_thresholds,)
  # features_shape = (num_features_anomaly_thresholds,)
  acc = tf.cast(x = tp + tn, dtype = tf.float64) \
    / tf.cast(x = tp + fn + fp + tn, dtype = tf.float64)
  pre = tf.cast(x = tp, dtype = tf.float64) / tf.cast(x = tp + fp, dtype = tf.float64)
  rec = tf.cast(x = tp, dtype = tf.float64) / tf.cast(x = tp + fn, dtype = tf.float64)
  f_beta_score = (1.0 + f_score_beta ** 2) * (pre * rec) / (f_score_beta ** 2 * pre + rec)

  return acc, pre, rec, f_beta_score

def find_best_anomaly_threshold(
  anomaly_thresholds, f_beta_score, user_passed_anomaly_threshold, anomaly_threshold_variable):
  if user_passed_anomaly_threshold == None:
    best_anomaly_threshold = tf.gather(
      params = anomaly_thresholds, 
      indices = tf.argmax(input = f_beta_score, 
      axis = 0)) # shape = ()
  else:
    best_anomaly_threshold = user_passed_anomaly_threshold # shape = ()

  with tf.control_dependencies(
    control_inputs = [
      tf.assign(ref = anomaly_threshold_variable, value = best_anomaly_threshold)]):
    return tf.identity(input = anomaly_threshold_variable)

# Create our model function to be used in our custom estimator
def lstm_encoder_decoder_autoencoder_anomaly_detection(features, labels, mode, params):
  print("\nlstm_encoder_decoder_autoencoder_anomaly_detection: features = \n{}".format(features))
  print("lstm_encoder_decoder_autoencoder_anomaly_detection: labels = \n{}".format(labels))
  print("lstm_encoder_decoder_autoencoder_anomaly_detection: mode = \n{}".format(mode))
  print("lstm_encoder_decoder_autoencoder_anomaly_detection: params = \n{}".format(params))

  # 0. Get input sequence tensor into correct shape
  # Get dynamic batch size in case there was a partially filled batch
  cur_batch_size = tf.shape(input = features[UNLABELED_CSV_COLUMNS[0]], out_type = tf.int64)[0]

  # Get the number of features 
  num_features = len(UNLABELED_CSV_COLUMNS)

  # Stack all of the features into a 3-D tensor
  # shape = (cur_batch_size, seq_len, num_features)
  X = tf.stack(values = [features[key] for key in UNLABELED_CSV_COLUMNS], axis = 2)

  # Unstack all of 3-D features tensor into a sequence(list) of 2-D tensors of 
  # shape = (cur_batch_size, num_features)
  X_sequence = tf.unstack(value = X, num = params["seq_len"], axis = 1)

  # Since this is an autoencoder, the features are the labels. 
  # It often works better though to have the labels in reverse order
  if params["reverse_labels_sequence"] == True:
    Y = tf.reverse_sequence(
      input = X,  # shape = (cur_batch_size, seq_len, num_features)
      seq_lengths = tf.tile(
        input = tf.constant(value = [params["seq_len"]], dtype = tf.int64), 
        multiples = tf.expand_dims(input = cur_batch_size, axis = 0)), 
      seq_axis = 1, 
      batch_axis = 0)
  else:
    Y = X  # shape = (cur_batch_size, seq_len, num_features)
  
  ################################################################################
  
  # 1. Create encoder of encoder-decoder LSTM stacks
  
  # Create our decoder now
  decoder_stacked_lstm_cells = create_LSTM_stack(
    params["decoder_lstm_hidden_units"], params["lstm_dropout_output_keep_probs"])
  
  # Create the encoder variable scope
  with tf.variable_scope("encoder"):
    # Create separate encoder cells with their own weights separate from decoder
    encoder_stacked_lstm_cells = create_LSTM_stack(
      params["encoder_lstm_hidden_units"], params["lstm_dropout_output_keep_probs"])

    # Encode the input sequence using our encoder stack of LSTMs
    # encoder_outputs = seq_len long of shape = (cur_batch_size, encoder_lstm_hidden_units[-1])
    # encoder_states = tuple of final encoder c_state and h_state for each layer
    encoder_outputs, encoder_states = tf.nn.static_rnn(
      cell = encoder_stacked_lstm_cells, 
      inputs = X_sequence, 
      initial_state = encoder_stacked_lstm_cells.zero_state(
        batch_size = tf.cast(x = cur_batch_size, dtype = tf.int32), 
        dtype = tf.float64), 
      dtype = tf.float64)

    # We just pass on the final c and h states of the encoder"s last layer, 
    # so extract that and drop the others
    # LSTMStateTuple shape = (cur_batch_size, lstm_hidden_units[-1])
    encoder_final_states = encoder_states[-1]

    # Extract the c and h states from the tuple
    # both have shape = (cur_batch_size, lstm_hidden_units[-1])
    encoder_final_c, encoder_final_h = encoder_final_states

    # In case the decoder"s first layer"s number of units is different than encoder's last 
    # layer's number of units, use a dense layer to map to the correct shape
    encoder_final_c_dense = tf.layers.dense(
      inputs = encoder_final_c, 
      units = params["decoder_lstm_hidden_units"][0], 
      activation = None) # shape = (cur_batch_size, decoder_lstm_hidden_units[0])
    encoder_final_h_dense = tf.layers.dense(
      inputs = encoder_final_h, 
      units = params["decoder_lstm_hidden_units"][0], 
      activation = None) # shape = (cur_batch_size, decoder_lstm_hidden_units[0])

    # The decoder"s first layer"s state comes from the encoder, 
    # the rest of the layers" initial states are zero
    decoder_intial_states = tuple(
      [tf.contrib.rnn.LSTMStateTuple(c = encoder_final_c_dense, h = encoder_final_h_dense)] + \
      [tf.contrib.rnn.LSTMStateTuple(
        c = tf.zeros(shape = [cur_batch_size, units], dtype = tf.float64), 
        h = tf.zeros(shape = [cur_batch_size, units], dtype = tf.float64)) 
      for units in params["decoder_lstm_hidden_units"][1:]])
  
  ################################################################################

  # 2. Create decoder of encoder-decoder LSTM stacks
  
  # Train our decoder now
  
  # Encoder-decoders work differently during training/evaluation and inference 
  # so we will have two separate subgraphs for each
  if mode == tf.estimator.ModeKeys.TRAIN and params["evaluation_mode"] == "reconstruction":
    # Break 3-D labels tensor into a list of 2-D tensors of shape = (cur_batch_size, num_features)
    unstacked_labels = tf.unstack(value = Y, num = params["seq_len"], axis = 1)

    # Call our decoder using the labels as our inputs, the encoder final state as our 
    # initial state, our other LSTM stack as our cells, and inference set to false
    decoder_outputs, decoder_states = rnn_decoder(
      decoder_inputs = unstacked_labels, 
      initial_state = decoder_intial_states, 
      cell = decoder_stacked_lstm_cells, 
      inference = False,
      dnn_hidden_units = params["dnn_hidden_units"],
      num_features = num_features)
  else:
    # Since this is inference create fake labels. The list length needs to be the output 
    # sequence length even though only the first element is the only one actually used 
    # (as our go signal)
    fake_labels = [tf.zeros(shape = [cur_batch_size, num_features], dtype = tf.float64) 
      for _ in range(params["seq_len"])]
    
    # Call our decoder using fake labels as our inputs, the encoder final state as our initial 
    # state, our other LSTM stack as our cells, and inference set to true
    # decoder_outputs = seq_len long of shape = (cur_batch_size, decoder_lstm_hidden_units[-1])
    # decoder_states = tuple of final decoder c_state and h_state for each layer
    decoder_outputs, decoder_states = rnn_decoder(
      decoder_inputs = fake_labels, 
      initial_state = decoder_intial_states, 
      cell = decoder_stacked_lstm_cells, 
      inference = True,
      dnn_hidden_units = params["dnn_hidden_units"],
      num_features = num_features)
  
  # Stack together the list of rank 2 decoder output tensors into one rank 3 tensor of
  # shape = (cur_batch_size, seq_len, lstm_hidden_units[-1])
  stacked_decoder_outputs = tf.stack(values = decoder_outputs, axis = 1)
  
  # Reshape rank 3 decoder outputs into rank 2 by folding sequence length into batch size
  # shape = (cur_batch_size * seq_len, lstm_hidden_units[-1])
  reshaped_stacked_decoder_outputs = tf.reshape(
    tensor = stacked_decoder_outputs, 
    shape = [cur_batch_size * params["seq_len"], params["decoder_lstm_hidden_units"][-1]])

  ################################################################################
  
  # 3. Create the DNN structure now after the encoder-decoder LSTM stack
  # Create the input layer to our DNN
  # shape = (cur_batch_size * seq_len, lstm_hidden_units[-1])
  network = reshaped_stacked_decoder_outputs
  
  # Reuse the same variable scope as we used within our decoder (for inference)
  with tf.variable_scope(name_or_scope = "dnn", reuse = tf.AUTO_REUSE):
    # Add hidden layers with the given number of units/neurons per layer
    for units in params["dnn_hidden_units"]:
      network = tf.layers.dense(
        inputs = network, 
        units = units, 
        activation = tf.nn.relu) # shape = (cur_batch_size * seq_len, dnn_hidden_units[i])

    # Connect the final hidden layer to a dense layer with no activation to get the logits
    logits = tf.layers.dense(
      inputs = network, 
      units = num_features, 
      activation = None) # shape = (cur_batch_size * seq_len, num_features)
  
  # Now that we are through the final DNN for each sequence element for each example in the batch,
  # reshape the predictions to match our labels.
  # shape = (cur_batch_size, seq_len, num_features)
  predictions = tf.reshape(
    tensor = logits, 
    shape = [cur_batch_size, params["seq_len"], num_features])
  
  # Variables for calculating error distribution statistics
  with tf.variable_scope(
    name_or_scope = "mahalanobis_distance_variables", reuse = tf.AUTO_REUSE):
    # Time based
    abs_err_count_time_variable = tf.get_variable(
      name = "abs_err_count_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    abs_err_mean_time_variable = tf.get_variable(
      name = "abs_err_mean_time_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [num_features], dtype = tf.float64),
      trainable = False) # shape = (num_features,)

    abs_err_cov_time_variable = tf.get_variable(
      name = "abs_err_cov_time_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [num_features, num_features], dtype = tf.float64),
      trainable = False) # shape = (num_features, num_features)

    abs_err_inv_cov_time_variable = tf.get_variable(
      name = "abs_err_inv_cov_time_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [num_features, num_features], dtype = tf.float64),
      trainable = False) # shape = (num_features, num_features)

    # Features based
    abs_err_count_features_variable = tf.get_variable(
      name = "abs_err_count_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    abs_err_mean_features_variable = tf.get_variable(
      name = "abs_err_mean_features_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [params["seq_len"]], dtype = tf.float64),
      trainable = False) # shape = (seq_len,)

    abs_err_cov_features_variable = tf.get_variable(
      name = "abs_err_cov_features_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [params["seq_len"], params["seq_len"]], dtype = tf.float64),
      trainable = False) # shape = (seq_len, seq_len)

    abs_err_inv_cov_features_variable = tf.get_variable(
      name = "abs_err_inv_cov_features_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [params["seq_len"], params["seq_len"]], dtype = tf.float64),
      trainable = False) # shape = (seq_len, seq_len)
  
  # Variables for automatically tuning anomaly thresholds
  with tf.variable_scope(
    name_or_scope = "mahalanobis_distance_threshold_variables", reuse = tf.AUTO_REUSE):
    # Time based
    tp_at_thresholds_time_variable = tf.get_variable(
      name = "tp_at_thresholds_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_time_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_time_anomaly_thresholds,)

    fn_at_thresholds_time_variable = tf.get_variable(
      name = "fn_at_thresholds_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_time_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_time_anomaly_thresholds,)

    fp_at_thresholds_time_variable = tf.get_variable(
      name = "fp_at_thresholds_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_time_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_time_anomaly_thresholds,)

    tn_at_thresholds_time_variable = tf.get_variable(
      name = "tn_at_thresholds_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_time_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_time_anomaly_thresholds,)

    time_anomaly_threshold_variable = tf.get_variable(
      name = "time_anomaly_threshold_variable",
      dtype = tf.float64,
      initializer = tf.zeros(shape = [], dtype = tf.float64),
      trainable = False) # shape = ()

    # Features based
    tp_at_thresholds_features_variable = tf.get_variable(
      name = "tp_at_thresholds_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_features_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_features_anomaly_thresholds,)

    fn_at_thresholds_features_variable = tf.get_variable(
      name = "fn_at_thresholds_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_features_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_features_anomaly_thresholds,)

    fp_at_thresholds_features_variable = tf.get_variable(
      name = "fp_at_thresholds_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_features_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_features_anomaly_thresholds,)

    tn_at_thresholds_features_variable = tf.get_variable(
      name = "tn_at_thresholds_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [params["num_features_anomaly_thresholds"]], dtype = tf.int64),
      trainable = False) # shape = (num_features_anomaly_thresholds,)

    features_anomaly_threshold_variable = tf.get_variable(
      name = "features_anomaly_threshold_variable", # shape = ()
      dtype = tf.float64,
      initializer = tf.zeros(shape = [], dtype = tf.float64),
      trainable = False)

  # Variables for automatically tuning anomaly thresholds
  with tf.variable_scope(
    name_or_scope = "anomaly_threshold_eval_variables", reuse = tf.AUTO_REUSE):
    # Time based
    tp_at_threshold_eval_time_variable = tf.get_variable(
      name = "tp_at_threshold_eval_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    fn_at_threshold_eval_time_variable = tf.get_variable(
      name = "fn_at_threshold_eval_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    fp_at_threshold_eval_time_variable = tf.get_variable(
      name = "fp_at_threshold_eval_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    tn_at_threshold_eval_time_variable = tf.get_variable(
      name = "tn_at_threshold_eval_time_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    # Features based
    tp_at_threshold_eval_features_variable = tf.get_variable(
      name = "tp_at_threshold_eval_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    fn_at_threshold_eval_features_variable = tf.get_variable(
      name = "fn_at_threshold_eval_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    fp_at_threshold_eval_features_variable = tf.get_variable(
      name = "fp_at_threshold_eval_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

    tn_at_threshold_eval_features_variable = tf.get_variable(
      name = "tn_at_threshold_eval_features_variable",
      dtype = tf.int64,
      initializer = tf.zeros(shape = [], dtype = tf.int64),
      trainable = False) # shape = ()

  dummy_variable = tf.get_variable(
    name = "dummy_variable",
    dtype = tf.float64,
    initializer = tf.zeros(shape = [], dtype = tf.float64),
    trainable = True) # shape = ()
  
  # Now branch off based on which mode we are in
  predictions_dict = None
  loss = None
  train_op = None
  eval_metric_ops = None
  export_outputs = None
  
  # 3. Loss function, training/eval ops
  if mode == tf.estimator.ModeKeys.TRAIN and params["evaluation_mode"] != "tune_anomaly_thresholds":
    if params["evaluation_mode"] == "reconstruction":
      loss = tf.losses.mean_squared_error(labels = Y, predictions = predictions)

      train_op = tf.contrib.layers.optimize_loss(
        loss = loss,
        global_step = tf.train.get_global_step(),
        learning_rate = params["learning_rate"],
        optimizer = "Adam")
    elif params["evaluation_mode"] == "calculate_error_distribution_statistics":
      error = Y - predictions # shape = (cur_batch_size, seq_len, num_features)
      
      absolute_error = tf.abs(x = error) # shape = (cur_batch_size, seq_len, num_features)

      ################################################################################

      with tf.variable_scope(
        name_or_scope = "mahalanobis_distance_variables", reuse = tf.AUTO_REUSE):
        # Check if batch is a singleton or not, very important for covariance math
        
        # Time based ########################################
        # shape = (cur_batch_size * seq_len, num_features)
        abs_err_reshaped_time = tf.reshape(
          tensor = absolute_error, 
          shape = [cur_batch_size * params["seq_len"], num_features])
        
        singleton_time_condition = tf.equal(
          x = cur_batch_size * params["seq_len"], y = 1) # shape = ()
        
        cov_time_update_op, mean_time_update_op, count_time_update_op, inv_time_update_op = tf.cond(
          pred = singleton_time_condition, 
          true_fn = lambda: singleton_batch_mahalanobis_distance_variable_updating(
            params["seq_len"], 
            abs_err_reshaped_time, 
            abs_err_count_time_variable, 
            abs_err_mean_time_variable, 
            abs_err_cov_time_variable, 
            abs_err_inv_cov_time_variable, 
            params["eps"]), 
          false_fn = lambda: non_singleton_batch_mahalanobis_distance_variable_updating(
            cur_batch_size, 
            params["seq_len"], 
            abs_err_reshaped_time, 
            abs_err_count_time_variable, 
            abs_err_mean_time_variable, 
            abs_err_cov_time_variable, 
            abs_err_inv_cov_time_variable,
            params["eps"]))

        # Features based ########################################
        # shape = (cur_batch_size, num_features, seq_len)
        abs_err_transposed_features = tf.transpose(a = absolute_error, perm = [0, 2, 1])

        # shape = (cur_batch_size * num_features, seq_len)
        abs_err_reshaped_features = tf.reshape(
          tensor = abs_err_transposed_features, 
          shape = [cur_batch_size * num_features, params["seq_len"]])

        # shape = ()
        singleton_features_condition = tf.equal(x = cur_batch_size * num_features, y = 1)
        
        cov_features_update_op, mean_features_update_op, count_features_update_op, inv_features_update_op = tf.cond(
          pred = singleton_features_condition, 
          true_fn = lambda: singleton_batch_mahalanobis_distance_variable_updating(
            num_features, 
            abs_err_reshaped_features, 
            abs_err_count_features_variable, 
            abs_err_mean_features_variable, 
            abs_err_cov_features_variable, 
            abs_err_inv_cov_features_variable,
            params["eps"]), 
          false_fn = lambda: non_singleton_batch_mahalanobis_distance_variable_updating(
            cur_batch_size, 
            num_features, 
            abs_err_reshaped_features, 
            abs_err_count_features_variable, 
            abs_err_mean_features_variable, 
            abs_err_cov_features_variable, 
            abs_err_inv_cov_features_variable,
            params["eps"]))

      # Lastly use control dependencies around loss to enforce the mahalanobis variables to be 
      # assigned, the control order matters, hence the separate contexts
      with tf.control_dependencies(control_inputs = [cov_time_update_op, cov_features_update_op]):
        with tf.control_dependencies(control_inputs = [mean_time_update_op, mean_features_update_op]):
          with tf.control_dependencies(control_inputs = [count_time_update_op, count_features_update_op]):
            with tf.control_dependencies(control_inputs = [inv_time_update_op, inv_features_update_op]):
              loss = tf.reduce_sum(input_tensor = tf.zeros(shape = (), dtype = tf.float64) * dummy_variable)

              train_op = tf.contrib.layers.optimize_loss(
                loss = loss,
                global_step = tf.train.get_global_step(),
                learning_rate = params["learning_rate"],
                optimizer = "SGD")
  elif mode == tf.estimator.ModeKeys.EVAL and params["evaluation_mode"] != "tune_anomaly_thresholds":
    # Reconstruction loss on evaluation set
    loss = tf.losses.mean_squared_error(labels = Y, predictions = predictions)
    
    if params["evaluation_mode"] == "reconstruction":
      # Reconstruction eval metrics
      eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(labels = Y, predictions = predictions),
        "mae": tf.metrics.mean_absolute_error(labels = Y, predictions = predictions)
      }
  elif mode == tf.estimator.ModeKeys.PREDICT or ((mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL) and params["evaluation_mode"] == "tune_anomaly_thresholds"):
    error = Y - predictions # shape = (cur_batch_size, seq_len, num_features)
    absolute_error = tf.abs(x = error) # shape = (cur_batch_size, seq_len, num_features)
    
    with tf.variable_scope(name_or_scope = "mahalanobis_distance_variables", reuse = tf.AUTO_REUSE):
      # Time based
      # shape = (cur_batch_size * seq_len, num_features)
      abs_err_reshaped_time = tf.reshape(
        tensor = absolute_error,
        shape = [cur_batch_size * params["seq_len"], num_features])

      mahalanobis_distance_time = mahalanobis_distance(
        error_vectors_reshaped = abs_err_reshaped_time,
        mean_vector = abs_err_mean_time_variable, 
        inv_covariance = abs_err_inv_cov_time_variable, 
        final_shape = params["seq_len"]) # shape = (cur_batch_size, seq_len)

      # Features based
      abs_err_mapped_features = tf.map_fn(
        fn = lambda x: tf.transpose(a = absolute_error[x, :, :]),
        elems = tf.range(start = 0, limit = cur_batch_size, dtype = tf.int64), 
        dtype = tf.float64) # shape = (cur_batch_size, num_features, seq_len)

      # shape = (cur_batch_size * num_features, seq_len)
      abs_err_reshaped_features = tf.reshape(
        tensor = abs_err_mapped_features,
        shape = [cur_batch_size * num_features, params["seq_len"]])

      mahalanobis_distance_features = mahalanobis_distance(
        error_vectors_reshaped = abs_err_reshaped_features,
        mean_vector = abs_err_mean_features_variable, 
        inv_covariance = abs_err_inv_cov_features_variable,
        final_shape = num_features) # shape = (cur_batch_size, num_features)

    if mode != tf.estimator.ModeKeys.PREDICT:
      labels_normal_mask = tf.equal(x = labels, y = 0)
      labels_anomalous_mask = tf.equal(x = labels, y = 1)
      
      if mode == tf.estimator.ModeKeys.TRAIN:
        with tf.variable_scope(
          name_or_scope = "mahalanobis_distance_variables", reuse = tf.AUTO_REUSE):
          # Time based
          # shape = (num_time_anomaly_thresholds,)
          time_anomaly_thresholds = tf.linspace(
            start = tf.constant(value = params["min_time_anomaly_threshold"], dtype = tf.float64),
            stop = tf.constant(value = params["max_time_anomaly_threshold"], dtype = tf.float64), 
            num = params["num_time_anomaly_thresholds"])

          tp_time_update_op, fn_time_update_op, fp_time_update_op, tn_time_update_op = \
            update_anomaly_threshold_variables(
              labels_normal_mask, 
              labels_anomalous_mask, 
              params["num_time_anomaly_thresholds"], 
              time_anomaly_thresholds, 
              mahalanobis_distance_time, 
              tp_at_thresholds_time_variable, 
              fn_at_thresholds_time_variable, 
              fp_at_thresholds_time_variable, 
              tn_at_thresholds_time_variable,
              mode)

          # Features based
          # shape = (num_features_anomaly_thresholds,)
          features_anomaly_thresholds = tf.linspace(
            start = tf.constant(value = params["min_features_anomaly_threshold"], dtype = tf.float64),
            stop = tf.constant(value = params["max_features_anomaly_threshold"], dtype = tf.float64), 
            num = params["num_features_anomaly_thresholds"])

          tp_features_update_op, fn_features_update_op, fp_features_update_op, tn_features_update_op = \
            update_anomaly_threshold_variables(
              labels_normal_mask, 
              labels_anomalous_mask, 
              params["num_features_anomaly_thresholds"], 
              features_anomaly_thresholds, 
              mahalanobis_distance_features, 
              tp_at_thresholds_features_variable, 
              fn_at_thresholds_features_variable, 
              fp_at_thresholds_features_variable, 
              tn_at_thresholds_features_variable, 
              mode)

        # Reconstruction loss on evaluation set
        with tf.control_dependencies(
          control_inputs = [
            tp_time_update_op, 
            fn_time_update_op, 
            fp_time_update_op, 
            tn_time_update_op, 
            tp_features_update_op, 
            fn_features_update_op, 
            fp_features_update_op, 
            tn_features_update_op]):
          # Time based
          acc_time, pre_time, rec_time, f_beta_score_time = \
            calculate_composite_classification_metrics(
              time_anomaly_thresholds, 
              tp_at_thresholds_time_variable, 
              fn_at_thresholds_time_variable, 
              fp_at_thresholds_time_variable, 
              tn_at_thresholds_time_variable,
              params["f_score_beta"])

          # Features based
          acc_features, pre_features, rec_features, f_beta_score_features = \
            calculate_composite_classification_metrics(
              features_anomaly_thresholds, 
              tp_at_thresholds_features_variable, 
              fn_at_thresholds_features_variable, 
              fp_at_thresholds_features_variable, 
              tn_at_thresholds_features_variable,
              params["f_score_beta"])

          with tf.control_dependencies(
            control_inputs = [pre_time, pre_features]):
            with tf.control_dependencies(
              control_inputs = [rec_time, rec_features]):
              with tf.control_dependencies(
                control_inputs = [f_beta_score_time, f_beta_score_features]):
                # Time based
                best_anomaly_threshold_time = find_best_anomaly_threshold(
                  time_anomaly_thresholds, 
                  f_beta_score_time, 
                  params["time_anomaly_threshold"], 
                  time_anomaly_threshold_variable)

                # Features based
                best_anomaly_threshold_features = find_best_anomaly_threshold(
                  features_anomaly_thresholds, 
                  f_beta_score_features, 
                  params["features_anomaly_threshold"], 
                  features_anomaly_threshold_variable)

                with tf.control_dependencies(
                  control_inputs = [
                    tf.assign(
                      ref = time_anomaly_threshold_variable, 
                      value = best_anomaly_threshold_time), 
                    tf.assign(ref = 
                              features_anomaly_threshold_variable, 
                              value = best_anomaly_threshold_features)]):
                  
                  loss = tf.reduce_sum(
                    input_tensor = tf.zeros(shape = (), dtype = tf.float64) * dummy_variable)

                  train_op = tf.contrib.layers.optimize_loss(
                    loss = loss,
                    global_step = tf.train.get_global_step(),
                    learning_rate = params["learning_rate"],
                    optimizer = "SGD")
      elif mode == tf.estimator.ModeKeys.EVAL:
        with tf.variable_scope(
          name_or_scope = "anomaly_threshold_eval_variables", reuse = tf.AUTO_REUSE):
          # Time based
          tp_time_update_op, fn_time_update_op, fp_time_update_op, tn_time_update_op = \
            update_anomaly_threshold_variables(
              labels_normal_mask, 
              labels_anomalous_mask, 
              1,
              time_anomaly_threshold_variable, 
              mahalanobis_distance_time, 
              tp_at_threshold_eval_time_variable, 
              fn_at_threshold_eval_time_variable, 
              fp_at_threshold_eval_time_variable, 
              tn_at_threshold_eval_time_variable,
              mode)

          # Features based
          tp_features_update_op, fn_features_update_op, fp_features_update_op, tn_features_update_op = \
            update_anomaly_threshold_variables(
              labels_normal_mask, 
              labels_anomalous_mask, 
              1,
              features_anomaly_threshold_variable, 
              mahalanobis_distance_features, 
              tp_at_threshold_eval_features_variable, 
              fn_at_threshold_eval_features_variable, 
              fp_at_threshold_eval_features_variable, 
              tn_at_threshold_eval_features_variable,
              mode)
          
        with tf.variable_scope(
          name_or_scope = "anomaly_threshold_eval_variables", reuse = tf.AUTO_REUSE):
          # Time based
          acc_time_update_op, pre_time_update_op, rec_time_update_op, f_beta_score_time_update_op = \
            calculate_composite_classification_metrics(
              time_anomaly_threshold_variable, 
              tp_at_threshold_eval_time_variable, 
              fn_at_threshold_eval_time_variable, 
              fp_at_threshold_eval_time_variable, 
              tn_at_threshold_eval_time_variable,
              params["f_score_beta"]) 

          # Features based
          acc_features_update_op, pre_features_update_op, rec_features_update_op, f_beta_score_features_update_op = \
            calculate_composite_classification_metrics(
              features_anomaly_threshold_variable, 
              tp_at_threshold_eval_features_variable, 
              fn_at_threshold_eval_features_variable, 
              fp_at_threshold_eval_features_variable, 
              tn_at_threshold_eval_features_variable,
              params["f_score_beta"]) 

        loss = tf.losses.mean_squared_error(labels = Y, predictions = predictions)
        
        acc_at_threshold_eval_time_variable = (tp_at_threshold_eval_time_variable + tn_at_threshold_eval_time_variable) / (tp_at_threshold_eval_time_variable + fn_at_threshold_eval_time_variable + fp_at_threshold_eval_time_variable + tn_at_threshold_eval_time_variable)
        pre_at_threshold_eval_time_variable = tp_at_threshold_eval_time_variable / (tp_at_threshold_eval_time_variable + fp_at_threshold_eval_time_variable)
        rec_at_threshold_eval_time_variable = tp_at_threshold_eval_time_variable / (tp_at_threshold_eval_time_variable + fn_at_threshold_eval_time_variable)
        f_beta_score_at_threshold_eval_time_variable = (1.0 + params["f_score_beta"] ** 2) * pre_at_threshold_eval_time_variable * rec_at_threshold_eval_time_variable / (params["f_score_beta"] ** 2 * pre_at_threshold_eval_time_variable + rec_at_threshold_eval_time_variable)
        
        acc_at_threshold_eval_features_variable = (tp_at_threshold_eval_features_variable + tn_at_threshold_eval_features_variable) / (tp_at_threshold_eval_features_variable + fn_at_threshold_eval_features_variable + fp_at_threshold_eval_features_variable + tn_at_threshold_eval_features_variable)
        pre_at_threshold_eval_features_variable = tp_at_threshold_eval_features_variable / (tp_at_threshold_eval_features_variable + fp_at_threshold_eval_features_variable)
        rec_at_threshold_eval_features_variable = tp_at_threshold_eval_features_variable / (tp_at_threshold_eval_features_variable + fn_at_threshold_eval_features_variable)
        f_beta_score_at_threshold_eval_features_variable = (1.0 + params["f_score_beta"] ** 2) * pre_at_threshold_eval_features_variable * rec_at_threshold_eval_features_variable / (params["f_score_beta"] ** 2 * pre_at_threshold_eval_features_variable + rec_at_threshold_eval_features_variable)

        # Anomaly detection eval metrics
        eval_metric_ops = {
          # Time based
          "time_anomaly_tp": (tp_at_threshold_eval_time_variable, tp_time_update_op),
          "time_anomaly_fn": (fn_at_threshold_eval_time_variable, fn_time_update_op),
          "time_anomaly_fp": (fp_at_threshold_eval_time_variable, fp_time_update_op),
          "time_anomaly_tn": (tn_at_threshold_eval_time_variable, tn_time_update_op),

          "time_anomaly_acc": (acc_at_threshold_eval_time_variable, acc_time_update_op),
          "time_anomaly_pre": (pre_at_threshold_eval_time_variable, pre_time_update_op),
          "time_anomaly_rec": (rec_at_threshold_eval_time_variable, rec_time_update_op),
          "time_anomaly_f_beta_score": (f_beta_score_at_threshold_eval_time_variable, f_beta_score_time_update_op),

           # Features based
          "features_anomaly_tp": (tp_at_threshold_eval_features_variable, tp_features_update_op),
          "features_anomaly_fn": (fn_at_threshold_eval_features_variable, fn_features_update_op),
          "features_anomaly_fp": (fp_at_threshold_eval_features_variable, fp_features_update_op),
          "features_anomaly_tn": (tn_at_threshold_eval_features_variable, tn_features_update_op),

          "features_anomaly_acc": (acc_at_threshold_eval_features_variable, acc_features_update_op),
          "features_anomaly_pre": (pre_at_threshold_eval_features_variable, pre_features_update_op),
          "features_anomaly_rec": (rec_at_threshold_eval_features_variable, rec_features_update_op),
          "features_anomaly_f_beta_score": (f_beta_score_at_threshold_eval_features_variable, f_beta_score_features_update_op)
        }
    else: # mode == tf.estimator.ModeKeys.PREDICT
      # Flag predictions as either normal or anomalous
      time_anomaly_flags = tf.where(
        condition = tf.reduce_any(
          input_tensor = tf.greater(
            x = tf.abs(x = mahalanobis_distance_time),
            y = time_anomaly_threshold_variable), 
          axis = 1), 
        x = tf.ones(shape = [cur_batch_size], dtype = tf.int64), 
        y = tf.zeros(shape = [cur_batch_size], dtype = tf.int64)) # shape = (cur_batch_size,)

      features_anomaly_flags = tf.where(
        condition = tf.reduce_any(
          input_tensor = tf.greater(
            x = tf.abs(x = mahalanobis_distance_features),
            y = features_anomaly_threshold_variable), 
          axis = 1), 
        x = tf.ones(shape = [cur_batch_size], dtype = tf.int64), 
        y = tf.zeros(shape = [cur_batch_size], dtype = tf.int64)) # shape = (cur_batch_size,)
    
      # Create predictions dictionary
      predictions_dict = {
        "Y": Y,
        "predictions": predictions, 
        "error": error,
        "absolute_error": absolute_error,
        "mahalanobis_distance_time": mahalanobis_distance_time, 
        "mahalanobis_distance_features": mahalanobis_distance_features, 
        "time_anomaly_flags": time_anomaly_flags, 
        "features_anomaly_flags": features_anomaly_flags}

      # Create export outputs
      export_outputs = {
        "predict_export_outputs": tf.estimator.export.PredictOutput(
          outputs = predictions_dict)}

  # Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
    mode = mode,
    predictions = predictions_dict,
    loss = loss,
    train_op = train_op,
    eval_metric_ops = eval_metric_ops,
    export_outputs = export_outputs)

# Create our serving input function to accept the data at serving and send it in the 
# right format to our custom estimator
def serving_input_fn(seq_len):
    # This function fixes the shape and type of our input strings
    def fix_shape_and_type_for_serving(placeholder):
        current_batch_size = tf.shape(input = placeholder, out_type = tf.int64)[0]
        
        # String split each string in batch and output values from the resulting SparseTensors
        split_string = tf.stack(values = tf.map_fn( # shape = (batch_size, seq_len)
            fn = lambda x: tf.string_split(source = [placeholder[x]], delimiter = ',').values, 
            elems = tf.range(start = 0, limit = current_batch_size, dtype = tf.int64), 
            dtype = tf.string), axis = 0)
        
        # Convert each string in the split tensor to float
        # shape = (batch_size, seq_len)
        feature_tensor = tf.string_to_number(string_tensor = split_string, out_type = tf.float64)
        
        return feature_tensor
    
    # This function fixes dynamic shape ambiguity of last dimension so that we will be able to 
    # use it in our DNN (since tf.layers.dense require the last dimension to be known)
    def get_shape_and_set_modified_shape_2D(tensor, additional_dimension_sizes):
        # Get static shape for tensor and convert it to list
        shape = tensor.get_shape().as_list()
        # Set outer shape to additional_dimension_sizes[0] since know this is the correct size
        shape[1] = additional_dimension_sizes[0]
        # Set the shape of tensor to our modified shape
        tensor.set_shape(shape = shape) # shape = (batch_size, additional_dimension_sizes[0])

        return tensor
            
    # Create placeholders to accept the data sent to the model at serving time
    # All features come in as a batch of strings, shape = (batch_size,), 
    # this was so because of passing the arrays to online ml-engine prediction
    feature_placeholders = {
        feature: tf.placeholder(
          dtype = tf.string, shape = [None]) for feature in UNLABELED_CSV_COLUMNS
    }
    
    # Create feature tensors
    features = {key: fix_shape_and_type_for_serving(placeholder = tensor) 
      for key, tensor in feature_placeholders.items()}
    
    # Fix dynamic shape ambiguity of feature tensors for our DNN
    features = {key: get_shape_and_set_modified_shape_2D(
      tensor = tensor, additional_dimension_sizes = [seq_len]) 
                for key, tensor in features.items()}

    return tf.estimator.export.ServingInputReceiver(
      features = features, receiver_tensors = feature_placeholders)

# Create estimator to train and evaluate
def train_and_evaluate(args):
  # Create our custom estimator using our model function
  estimator = tf.estimator.Estimator(
    model_fn = lstm_encoder_decoder_autoencoder_anomaly_detection,
    model_dir = args["output_dir"],
    params = {
      "seq_len": args["seq_len"],
      "reverse_labels_sequence": args["reverse_labels_sequence"],
      "encoder_lstm_hidden_units": args["encoder_lstm_hidden_units"],
      "decoder_lstm_hidden_units": args["decoder_lstm_hidden_units"],
      "lstm_dropout_output_keep_probs": args["lstm_dropout_output_keep_probs"], 
      "dnn_hidden_units": args["dnn_hidden_units"], 
      "learning_rate": args["learning_rate"],
      "evaluation_mode": args["evaluation_mode"],
      "num_time_anomaly_thresholds": args["num_time_anomaly_thresholds"],
      "num_features_anomaly_thresholds": args["num_features_anomaly_thresholds"],
      "min_time_anomaly_threshold": args["min_time_anomaly_threshold"],
      "max_time_anomaly_threshold": args["max_time_anomaly_threshold"],
      "min_features_anomaly_threshold": args["min_features_anomaly_threshold"],
      "max_features_anomaly_threshold": args["max_features_anomaly_threshold"],
      "time_anomaly_threshold": args["time_anomaly_threshold"], 
      "features_anomaly_threshold": args["features_anomaly_threshold"],
      "eps": args["eps"],
      "f_score_beta": args["f_score_beta"]})
  
  if args["evaluation_mode"] == "reconstruction":
    early_stopping_hook = tf.contrib.estimator.stop_if_no_decrease_hook(
      estimator = estimator,
      metric_name = "rmse",
      max_steps_without_decrease = 100,
      min_steps = 1000,
      run_every_secs = 60,
      run_every_steps = None)

    # Create train spec to read in our training data
    train_spec = tf.estimator.TrainSpec(
      input_fn = read_dataset(
        filename = args["train_file_pattern"],
        mode = tf.estimator.ModeKeys.TRAIN, 
        batch_size = args["train_batch_size"],
        params = args),
      max_steps = args["train_steps"], 
      hooks = [early_stopping_hook])

    # Create eval spec to read in our validation data and export our model
    eval_spec = tf.estimator.EvalSpec(
      input_fn = read_dataset(
        filename = args["eval_file_pattern"], 
        mode = tf.estimator.ModeKeys.EVAL, 
        batch_size = args["eval_batch_size"],
        params = args),
      steps = None,
      start_delay_secs = args["start_delay_secs"], # start evaluating after N seconds
      throttle_secs = args["throttle_secs"])  # evaluate every N seconds

    # Create train and evaluate loop to train and evaluate our estimator
    tf.estimator.train_and_evaluate(
      estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)
  else:
    if args["evaluation_mode"] == "calculate_error_distribution_statistics":
      # Get final mahalanobis statistics over the entire validation_1 dataset
      train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset(
          filename = args["train_file_pattern"],
          mode = tf.estimator.ModeKeys.EVAL, # only read through validation dataset once
          batch_size = args["train_batch_size"],
          params = args),
        max_steps = args["train_steps"])

      # Don't create exporter for serving yet since anomaly thresholds aren't trained yet
      exporter = None
    elif args["evaluation_mode"] == "tune_anomaly_thresholds":
      # Tune anomaly thresholds using valdiation_2 and validation_anomaly datasets
      train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset(
          filename = args["train_file_pattern"],
          mode = tf.estimator.ModeKeys.EVAL, # only read through validation dataset once
          batch_size = args["train_batch_size"],
          params = args),
        max_steps = args["train_steps"])
      
      # Create exporter that uses serving_input_fn to create saved_model for serving
      exporter = tf.estimator.LatestExporter(
        name = "exporter", serving_input_receiver_fn = lambda: serving_input_fn(args["sequence_length"]))

    # Create eval spec to read in our validation data and export our model
    eval_spec = tf.estimator.EvalSpec(
      input_fn = read_dataset(
        filename = args["eval_file_pattern"], 
        mode = tf.estimator.ModeKeys.EVAL, 
        batch_size = args["eval_batch_size"],
        params = args),
      steps = None,
      exporters = exporter,
      start_delay_secs = args["start_delay_secs"], # start evaluating after N seconds
      throttle_secs = args["throttle_secs"])  # evaluate every N seconds
    
    # Create train and evaluate loop to train and evaluate our estimator
    tf.estimator.train_and_evaluate(
      estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)

In [ ]:
%%writefile lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer/task.py
import argparse
import json
import os

from . import model

import tensorflow as tf

if __name__ == "__main__":
  parser = argparse.ArgumentParser()
  # File arguments
  parser.add_argument(
    "--train_file_pattern",
    help = "GCS location to read training data",
    required = True
  )
  parser.add_argument(
    "--eval_file_pattern",
    help = "GCS location to read evaluation data",
    required = True
  )
  parser.add_argument(
    "--output_dir",
    help = "GCS location to write checkpoints and export models",
    required = True
  )
  parser.add_argument(
    "--job-dir",
    help = "this model ignores this field, but it is required by gcloud",
    default = "junk"
  )

  # Sequence shape hyperparameters
  parser.add_argument(
    "--seq_len",
    help = "Number of timesteps to include in each example",
    type = int,
    default = 32
  )
  parser.add_argument(
    "--horizon",
    help = "Number of timesteps to skip into the future",
    type = int,
    default = 0
  )
  parser.add_argument(
    "--reverse_labels_sequence",
    help = "Whether we should reverse the labels sequence dimension or not",
    type = bool,
    default = True
  )

  # Architecture hyperparameters

  # LSTM hyperparameters
  parser.add_argument(
    "--encoder_lstm_hidden_units",
    help = "Hidden layer sizes to use for LSTM encoder",
    default = "64 32 16"
  )
  parser.add_argument(
    "--decoder_lstm_hidden_units",
    help = "Hidden layer sizes to use for LSTM decoder",
    default = "16 32 64"
  )
  parser.add_argument(
    "--lstm_dropout_output_keep_probs",
    help = "Keep probabilties for LSTM outputs",
    default = "1.0 1.0 1.0"
  )

  # DNN hyperparameters
  parser.add_argument(
    "--dnn_hidden_units",
    help = "Hidden layer sizes to use for DNN",
    default = "1024 256 64"
  )

  # Training parameters
  parser.add_argument(
    "--train_batch_size",
    help = "Number of examples in training batch",
    type = int,
    default = 32
  )
  parser.add_argument(
    "--eval_batch_size",
    help = "Number of examples in evaluation batch",
    type = int,
    default = 32
  )
  parser.add_argument(
    "--train_steps",
    help = "Number of batches to train for",
    type = int,
    default = 2000
  )
  parser.add_argument(
    "--learning_rate",
    help = "The learning rate, how quickly or slowly we train our model by scaling the gradient",
    type = float,
    default = 0.1
  )
  parser.add_argument(
    "--start_delay_secs",
    help = "Number of seconds to wait before first evaluation",
    type = int,
    default = 60
  )
  parser.add_argument(
    "--throttle_secs",
    help = "Number of seconds to wait between evaluations",
    type = int,
    default = 120
  )

  # Anomaly detection
  parser.add_argument(
    "--evaluation_mode",
    help = "Which evaluation mode we are in (reconstruction, calculate_error_distribution_statistics, tune_anomaly_thresholds)",
    type = str,
    default = "reconstruction"
  )
  parser.add_argument(
    "--num_time_anomaly_thresholds",
    help = "Number of anomaly thresholds to evaluate in the time dimension",
    type = int,
    default = 120
  )
  parser.add_argument(
    "--num_features_anomaly_thresholds",
    help = "Number of anomaly thresholds to evaluate in the features dimension",
    type = int,
    default = 120
  )
  parser.add_argument(
    "--min_time_anomaly_threshold",
    help = "The minimum anomaly threshold to evaluate in the time dimension",
    type = float,
    default = 100.0
  )
  parser.add_argument(
    "--max_time_anomaly_threshold",
    help = "The maximum anomaly threshold to evaluate in the time dimension",
    type = float,
    default = 2000.0
  )
  parser.add_argument(
    "--min_features_anomaly_threshold",
    help = "The minimum anomaly threshold to evaluate in the time dimension",
    type = float,
    default = 100.0
  )
  parser.add_argument(
    "--max_features_anomaly_threshold",
    help = "The maximum anomaly threshold to evaluate in the time dimension",
    type = float,
    default = 2000.0
  )
  parser.add_argument(
    "--time_anomaly_threshold",
    help = "The anomaly threshold in the time dimension",
    type = float,
    default = None
  )
  parser.add_argument(
    "--features_anomaly_threshold",
    help = "The anomaly threshold in the features dimension",
    type = float,
    default = None
  )
  parser.add_argument(
    "--eps",
    help = "The precision value to add to the covariance matrix before inversion to avoid being singular",
    type = str,
    default = "1e-12"
  )
  parser.add_argument(
    "--f_score_beta",
    help = "The value of beta of the f-beta score",
    type = float,
    default = 0.05
  )

  # Parse all arguments
  args = parser.parse_args()
  arguments = args.__dict__

  # Unused args provided by service
  arguments.pop("job_dir", None)
  arguments.pop("job-dir", None)

  # Fix list arguments
  arguments["encoder_lstm_hidden_units"] = [int(x) 
                                            for x in arguments["encoder_lstm_hidden_units"].split(' ')]
  arguments["decoder_lstm_hidden_units"] = [int(x) 
                                            for x in arguments["decoder_lstm_hidden_units"].split(' ')]
  arguments["lstm_dropout_output_keep_probs"] = [float(x) 
                                                 for x in arguments["lstm_dropout_output_keep_probs"].split(' ')]
  arguments["dnn_hidden_units"] = [int(x) 
                                   for x in arguments["dnn_hidden_units"].split(' ')]

  # Fix eps argument
  arguments["eps"] = float(arguments["eps"])

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  arguments["output_dir"] = os.path.join(
      arguments["output_dir"],
      json.loads(
          os.environ.get("TF_CONFIG", "{}")
      ).get("task", {}).get("trial", "")
  )

  # Run the training job
  model.train_and_evaluate(arguments)

# Training model module

## Locally

### Train reconstruction variables

In [1]:
%%bash
rm -rf trained_model
export PYTHONPATH=$PYTHONPATH:$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module
python -m trainer.task \
  --train_file_pattern="data/training_normal_sequences.csv" \
  --eval_file_pattern="data/validation_normal_1_sequences.csv" \
  --output_dir=$PWD/trained_model \
  --job-dir=./tmp \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2000 \
  --learning_rate=0.1 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --evaluation_mode="reconstruction" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


lstm_encoder_decoder_autoencoder_anomaly_detection: features = 
{'tag_0': <tf.Tensor 'IteratorGetNext:0' shape=(?, 30) dtype=float64>, 'tag_1': <tf.Tensor 'IteratorGetNext:1' shape=(?, 30) dtype=float64>, 'tag_2': <tf.Tensor 'IteratorGetNext:2' shape=(?, 30) dtype=float64>, 'tag_3': <tf.Tensor 'IteratorGetNext:3' shape=(?, 30) dtype=float64>, 'tag_4': <tf.Tensor 'IteratorGetNext:4' shape=(?, 30) dtype=float64>}
lstm_encoder_decoder_autoencoder_anomaly_detection: labels = 
None
lstm_encoder_decoder_autoencoder_anomaly_detection: mode = 
train
lstm_encoder_decoder_autoencoder_anomaly_detection: params = 
{'max_batch_features_anomaly_threshold': 2000.0, 'min_batch_time_anomaly_threshold': 100.0, 'number_of_batch_time_anomaly_thresholds': 300, 'f_score_beta

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f382635fe90>, '_model_dir': '/home/jupyter/artificial_intelligence/machine_learning/anomaly_detection/tf_lstm_encoder_decoder_autoencoder/trained_model/', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Not using Distribu

### Train error distribution statistics variables

In [2]:
%%bash
export PYTHONPATH=$PYTHONPATH:$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module
python -m trainer.task \
  --train_file_pattern="data/validation_normal_1_sequences.csv" \
  --eval_file_pattern="data/validation_normal_1_sequences.csv" \
  --output_dir=$PWD/trained_model \
  --job-dir=./tmp \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2200 \
  --evaluation_mode="calculate_error_distribution_statistics" \
  --eps="1e-12" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300


lstm_encoder_decoder_autoencoder_anomaly_detection: features = 
{'tag_0': <tf.Tensor 'IteratorGetNext:0' shape=(?, 30) dtype=float64>, 'tag_1': <tf.Tensor 'IteratorGetNext:1' shape=(?, 30) dtype=float64>, 'tag_2': <tf.Tensor 'IteratorGetNext:2' shape=(?, 30) dtype=float64>, 'tag_3': <tf.Tensor 'IteratorGetNext:3' shape=(?, 30) dtype=float64>, 'tag_4': <tf.Tensor 'IteratorGetNext:4' shape=(?, 30) dtype=float64>}
lstm_encoder_decoder_autoencoder_anomaly_detection: labels = 
None
lstm_encoder_decoder_autoencoder_anomaly_detection: mode = 
train
lstm_encoder_decoder_autoencoder_anomaly_detection: params = 
{'max_batch_features_anomaly_threshold': 2000.0, 'min_batch_time_anomaly_threshold': 100.0, 'number_of_batch_time_anomaly_thresholds': 300, 'f_score_beta': 0.05, 'time_anomaly_threshold': None, 'learning_rate': 0.1, 'min_batch_features_anomaly_threshold': 100.0, 'decoder_lstm_hidden_units': [16, 32, 64], 'lstm_dropout_output_keep_probs': [0.9, 0.95, 1.0], 'dnn_hidden_units': [1024, 256,

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f345df26e10>, '_model_dir': '/home/jupyter/artificial_intelligence/machine_learning/anomaly_detection/tf_lstm_encoder_decoder_autoencoder/trained_model/', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Not using Distribu

### Tune anomaly thresholds

In [3]:
%%bash
export PYTHONPATH=$PYTHONPATH:$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module
python -m trainer.task \
  --train_file_pattern="data/labeled_validation_mixed_sequences.csv" \
  --eval_file_pattern="data/labeled_validation_mixed_sequences.csv" \
  --output_dir=$PWD/trained_model \
  --job-dir=./tmp \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2400 \
  --evaluation_mode="tune_anomaly_thresholds" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300 \
  --min_time_anomaly_threshold=1.0 \
  --max_time_anomaly_threshold=20.0 \
  --min_features_anomaly_threshold=20.0 \
  --max_features_anomaly_threshold=80.0 \
  --f_score_beta=0.05


lstm_encoder_decoder_autoencoder_anomaly_detection: features = 
{'tag_0': <tf.Tensor 'IteratorGetNext:0' shape=(?, 30) dtype=float64>, 'tag_1': <tf.Tensor 'IteratorGetNext:1' shape=(?, 30) dtype=float64>, 'tag_2': <tf.Tensor 'IteratorGetNext:2' shape=(?, 30) dtype=float64>, 'tag_3': <tf.Tensor 'IteratorGetNext:3' shape=(?, 30) dtype=float64>, 'tag_4': <tf.Tensor 'IteratorGetNext:4' shape=(?, 30) dtype=float64>}
lstm_encoder_decoder_autoencoder_anomaly_detection: labels = 
Tensor("IteratorGetNext:5", shape=(?,), dtype=float64, device=/device:CPU:0)
lstm_encoder_decoder_autoencoder_anomaly_detection: mode = 
train
lstm_encoder_decoder_autoencoder_anomaly_detection: params = 
{'max_batch_features_anomaly_threshold': 80.0, 'min_batch_time_anomaly_threshold': 1.0, 'number_of_batch_time_anomaly_thresholds': 300, 'f_score_beta': 0.05, 'time_anomaly_threshold': None, 'learning_rate': 0.1, 'min_batch_features_anomaly_threshold': 20.0, 'decoder_lstm_hidden_units': [16, 32, 64], 'lstm_dropout_ou

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f13a4b6ae10>, '_model_dir': '/home/jupyter/artificial_intelligence/machine_learning/anomaly_detection/tf_lstm_encoder_decoder_autoencoder/trained_model/', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}
INFO:tensorflow:Not using Distribu

## GCloud

Copy data over to bucket

In [6]:
%%bash
gsutil -m cp -r data/* gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data

Copying file://data/labeled_test_mixed_sequences.csv [Content-Type=text/csv]...
Copying file://data/labeled_validation_mixed_sequences.csv [Content-Type=text/csv]...
Copying file://data/training_normal_sequences.csv [Content-Type=text/csv]...
Copying file://data/validation_normal_1_sequences.csv [Content-Type=text/csv]...
\
Operation completed over 4 objects/158.3 MiB.                                    


### Train reconstruction variables

In [ ]:
%%bash
OUTDIR=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/trained_model
JOBNAME=job_lstm_encoder_decoder_autoencoder_anomaly_detection_reconstruction_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.13 \
  -- \
  --train_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/training_normal_sequences.csv \
  --eval_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/validation_normal_1_sequences.csv \
  --output_dir=$OUTDIR \
  --job-dir=$OUTDIR \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2000 \
  --learning_rate=0.1 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --evaluation_mode="reconstruction" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300

### Hyperparameter tuning of reconstruction hyperparameters

In [ ]:
%%writefile hyperparam_reconstruction.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: rmse
    goal: MINIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: encoder_lstm_hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32 16", "256 128 16", "64 64 64"]
    - parameterName: decoder_lstm_hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32 16", "256 128 16", "64 64 64"]
    - parameterName: lstm_dropout_output_keep_probs
      type: CATEGORICAL
      categoricalValues: ["0.9 1.0 1.0", "0.95 0.95 1.0", "0.95 0.95 0.95"]
    - parameterName: dnn_hidden_units
      type: CATEGORICAL
      categoricalValues: ["256 128 64", "256 128 16", "64 64 64"]
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.001
      maxValue: 0.1
      scaleType: UNIT_LINEAR_SCALE

In [ ]:
%%bash
OUTDIR=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/hyperparam_reconstruction
JOBNAME=job_lstm_encoder_decoder_autoencoder_anomaly_detection_hyperparam_reconstruction_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --config=hyperparam_reconstruction.yaml \
  --runtime-version=1.13 \
  -- \
  --train_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/training_normal_sequences.csv \
  --eval_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/validation_normal_1_sequences.csv \
  --output_dir=$OUTDIR \
  --job-dir=$OUTDIR \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2000 \
  --start_delay_secs=60 \
  --throttle_secs=120 \
  --evaluation_mode="reconstruction" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300

### Train error distribution variables

In [ ]:
%%bash
OUTDIR=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/trained_model
JOBNAME=job_lstm_encoder_decoder_autoencoder_anomaly_detection_calculate_error_distribution_statistics_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.13 \
  -- \
  --train_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/validation_normal_1_sequences.csv \
  --eval_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/validation_normal_1_sequences.csv \
  --output_dir=$OUTDIR \
  --job-dir=$OUTDIR \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2200 \
  --evaluation_mode="calculate_error_distribution_statistics" \
  --eps="1e-12" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300

### Tune anomaly thresholds

In [ ]:
%%bash
OUTDIR=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/trained_model
JOBNAME=job_lstm_encoder_decoder_autoencoder_anomaly_detection_tune_anomaly_thresholds_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/lstm_encoder_decoder_autoencoder_anomaly_detection_module/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.13 \
  -- \
  --train_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/labeled_validation_mixed_sequences.csv \
  --eval_file_pattern=gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/data/labeled_validation_mixed_sequences.csv \
  --output_dir=$OUTDIR \
  --job-dir=$OUTDIR \
  --seq_len=30 \
  --horizon=0 \
  --reverse_labels_sequence=True \
  --encoder_lstm_hidden_units="64 32 16" \
  --encoder_lstm_hidden_units="16 32 64" \
  --lstm_dropout_output_keep_probs="0.9 0.95 1.0" \
  --dnn_hidden_units="1024 256 64" \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --train_steps=2400 \
  --evaluation_mode="tune_anomaly_thresholds" \
  --num_time_anomaly_thresholds=300 \
  --num_features_anomaly_thresholds=300 \
  --min_time_anomaly_threshold=2.0 \
  --max_time_anomaly_threshold=15.0 \
  --min_features_anomaly_threshold=20 \
  --max_features_anomaly_threshold=60 \
  --f_score_beta=0.05

# Deploy

In [ ]:
%%bash
MODEL_NAME="lstm_autoencoder_anomaly_detection"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://$BUCKET/lstm_encoder_decoder_autoencoder_anomaly_detection/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create $MODEL_NAME --regions $REGION
gcloud ml-engine versions create $MODEL_VERSION --model $MODEL_NAME --origin $MODEL_LOCATION --runtime-version 1.13

# Prediction

In [42]:
UNLABELED_CSV_COLUMNS = ["tag_{0}".format(tag) for tag in range(0, 5)]

In [28]:
import numpy as np
labeled_test_mixed_sequences_array = np.loadtxt(fname = "data/labeled_test_mixed_sequences.csv", dtype = str, delimiter = ";")
print("labeled_test_mixed_sequences_array.shape = {}".format(labeled_test_mixed_sequences_array.shape))

labeled_test_mixed_sequences_array.shape = (12800, 6)


In [58]:
number_of_prediction_instances = 10
print("labels = {}".format(labeled_test_mixed_sequences_array[0:number_of_prediction_instances, -1]))

labels = ['1' '0' '0' '0' '0' '0' '1' '1' '1' '1']


### Local prediction from local model

In [59]:
with open('test_sequences.json', 'w') as outfile:
  test_data_normal_string_list = labeled_test_mixed_sequences_array.tolist()[0:number_of_prediction_instances]
  json_string = ""
  for example in test_data_normal_string_list:
    json_string += "{" + ','.join(["{0}: \"{1}\"".format('\"' + UNLABELED_CSV_COLUMNS[i] + '\"', example[i]) 
                                   for i in range(len(UNLABELED_CSV_COLUMNS))]) + "}\n"
  json_string = json_string.replace(' ', '').replace(':', ': ').replace(',', ', ')
  print(json_string)
  outfile.write("%s" % json_string)

{"tag_0": "0.52632093, 1.53113314, -0.38315299, -0.35204753, 1.78125985, 1.20516328, -0.93736758, 0.3218563, 1.54225703, -0.14213172, -0.46315873, 1.90137753, 1.30125425, -0.34557603, 0.04739493, 2.20711214, -0.36731206, -0.69530595, 1.42715761, 0.92510047, -0.93935246, -2.1040559, -30.57388136, -4.08496153, 6.56978318, 26.39129804, 12.28809184, -10.7489366, -4.36919712, 18.49367233", "tag_1": "0.46146062, 2.24852577, -0.17950931, -0.84713643, 1.29672667, 2.39650857, -0.54226883, -0.54039575, 1.566966, 1.55978347, -1.39289256, -0.27748181, 2.88539864, 0.39155215, -1.63806991, 1.2352758, 2.35941941, -0.16344794, -0.6027153, 1.32728367, 1.86455497, -11.76498463, -5.34179003, -32.52345842, -21.09403891, -24.85357558, 12.82416297, -36.35115724, -3.92281946, -13.16864548", "tag_2": "0.96228059, 2.44595857, 0.99453653, -1.12445684, -0.92328179, 1.49070788, 2.00492373, 0.43074297, -1.37442886, -0.12654038, 1.53854117, 1.01929267, -0.33746006, -0.61782692, 1.3150855, 2.09451795, 0.63695697, -0

In [60]:
%%bash
model_dir=$(ls ${PWD}/trained_model/export/exporter | tail -1)
gcloud ml-engine local predict \
  --model-dir=${PWD}/trained_model/export/exporter/${model_dir} \
  --json-instances=./test_sequences.json

Y                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

2019-04-25 08:44:04.166882: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000185000 Hz
2019-04-25 08:44:04.178775: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5597193516a0 executing computations on platform Host. Devices:
2019-04-25 08:44:04.178809: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-25 08:44:04.189329: I tensorflow/core/common_runtime/process_util.cc:71] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.

### GCloud ML-Engine prediction from deployed model

In [ ]:
test_data_normal_string_list = labeled_test_mixed_sequences_array.tolist()[0:number_of_prediction_instances]

In [61]:
# Format dataframe to instances list to get sent to ML-Engine
instances = [{UNLABELED_CSV_COLUMNS[i]: example[i]
              for i in range(len(UNLABELED_CSV_COLUMNS))} 
             for example in labeled_test_mixed_sequences_array.tolist()[0:number_of_prediction_instances]]

In [62]:
instances

[{'tag_0': '0.52632093,1.53113314,-0.38315299,-0.35204753,1.78125985,1.20516328,-0.93736758,0.3218563,1.54225703,-0.14213172,-0.46315873,1.90137753,1.30125425,-0.34557603,0.04739493,2.20711214,-0.36731206,-0.69530595,1.42715761,0.92510047,-0.93935246,-2.1040559,-30.57388136,-4.08496153,6.56978318,26.39129804,12.28809184,-10.7489366,-4.36919712,18.49367233',
  'tag_1': '0.46146062,2.24852577,-0.17950931,-0.84713643,1.29672667,2.39650857,-0.54226883,-0.54039575,1.566966,1.55978347,-1.39289256,-0.27748181,2.88539864,0.39155215,-1.63806991,1.2352758,2.35941941,-0.16344794,-0.6027153,1.32728367,1.86455497,-11.76498463,-5.34179003,-32.52345842,-21.09403891,-24.85357558,12.82416297,-36.35115724,-3.92281946,-13.16864548',
  'tag_2': '0.96228059,2.44595857,0.99453653,-1.12445684,-0.92328179,1.49070788,2.00492373,0.43074297,-1.37442886,-0.12654038,1.53854117,1.01929267,-0.33746006,-0.61782692,1.3150855,2.09451795,0.63695697,-0.4486449,-0.03834883,1.67575155,1.92891937,-3.04365741,-6.3389432,12.2

In [63]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials = credentials)

request_data = {"instances": instances}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'lstm_autoencoder_anomaly_detection', 'v1')
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {'predictions': [{'error': [[18.01220810834468, -13.57548733409824, 27.79412058105622, -5.247824909005001, -27.9074291005457], [-4.850661341655317, -4.32966131409824, -22.48047619894378, -8.904843809005, 7.872035939454301], [-11.230400821655318, -36.75799909409824, -12.364559208943778, 31.187207350995, 6.573984739454302], [11.806627618344683, 12.41732111590176, -14.520073488943778, 8.507819090995001, -7.044770980545698], [25.90983381834468, -25.26041743409824, 9.739070211056223, 6.4814154909949995, 22.6900423094543], [6.088318958344683, -21.500880764098238, -24.73608801894378, -20.146790749005, 18.5183761994543], [-4.566425751655317, -32.93030027409824, 11.779828151056222, -22.512573039004998, -3.4579930605456983], [-31.05534558165532, -5.7486318840982396, -6.804162458943778, 14.859617090995, 22.9101373094543], [-2.585520121655317, -12.17182648409824, -3.508876668943778, 1.759084730995, 1.7827575294543017], [-1.4208166816553172, 1.4577131159017602, 1.4637001110562218, 1.4119